<a href="https://colab.research.google.com/github/CISC452MLProject/CISC-452-Group-22/blob/main/group22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Image analysis


In [2]:
import os
import cv2
from mmdet.apis import inference_detector, init_detector
import requests
from mmpose.utils import adapt_mmdet_pipeline
from mmpose.registry import VISUALIZERS
from mmpose.apis import inference_topdown
from mmpose.apis import init_model as init_pose_estimator
import mmcv
import numpy as np
from mmpose.evaluation.functional import nms
from mmpose.structures import merge_data_samples

Gather the model weights and code


In [ ]:
#model configs
dec_checkpoint_url = "https://download.openmmlab.com/mmpose/v1/projects/rtmpose/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth"
pos_config_url = "https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-m_simcc-body7_pt-body7_420e-256x192-e48f03d0_20230504.pth"

#bbox detection
detector = init_detector(
        "mmpose/demo/mmdetection_cfg/rtmdet_m_640-8xb32_coco-person.py", dec_checkpoint_url, device="cuda")
detector.cfg = adapt_mmdet_pipeline(detector.cfg)


pose_estimator = init_pose_estimator(
        "mmpose/configs/body_2d_keypoint/rtmpose/body8/rtmpose-m_8xb256-420e_body8-256x192.py ",
        pos_config_url,
        device="cuda",
        cfg_options=dict(
            model=dict(test_cfg=dict(output_heatmaps=True))))

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmpose/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-m_simcc-body7_pt-body7_420e-256x192-e48f03d0_20230504.pth


In [ ]:
#setup visualizer
pose_estimator.cfg.visualizer.radius = 3
pose_estimator.cfg.visualizer.alpha = 0.8
pose_estimator.cfg.visualizer.line_width = 1
visualizer = VISUALIZERS.build(pose_estimator.cfg.visualizer)

visualizer.set_dataset_meta(
    pose_estimator.dataset_meta, skeleton_style="mmpose")

In [ ]:

def process_img(img, show=False):
 
    #detect bbox
    det_result = inference_detector(detector, img)
    pred_instance = det_result.pred_instances.cpu().numpy()
    bboxes = np.concatenate(
        (pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
    bboxes = bboxes[np.logical_and(pred_instance.labels == 0,
                                    pred_instance.scores > 0.3)]
    bboxes = bboxes[nms(bboxes, 0.3), :4]

    #use bbox and pose estimator to generate our results
    pose_results = inference_topdown(pose_estimator, img, bboxes)
    data_samples = merge_data_samples(pose_results) 

    #create visualizations
    #this function call is really slow and should be optimized in the future, but for now we utilize it to either generate visualizations or render the frame to compile to an output video
    visualizer.add_datasample(
                'result',
                img,
                data_sample=data_samples,
                draw_gt=False,
                draw_heatmap=False,
                draw_bbox=True,
                skeleton_style="mmpose",
                show=show,
                wait_time=1)
    return data_samples.get('pred_instances', None)


# Single Image example


In [ ]:

img = mmcv.imread('C:\\Users\\mikeG\\Documents\\school\\cisc-452\\CISC-452-Group-22\\img\\41_2019-10-05_1-22.jpg')
img = mmcv.bgr2rgb(img)
process_img(img, True)

c:\Users\mikeG\Documents\school\cisc-452\CISC-452-Group-22\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
c:\users\mikeg\documents\school\cisc-452\cisc-452-group-22\mmpose\mmpose\models\heads\coord_cls_heads\rtmcc_head.py:217: UserWarning: The predicted simcc values are normalized for visualization. This may cause discrepancy between the keypoint scores and the 1D heatmaps.
  warnings.warn('The predicted simcc values are normalized for '


<InstanceData(

    META INFORMATION

    DATA FIELDS
    bbox_scores: array([1., 1., 1.], dtype=float32)
    bboxes: array([[ 995.2291 ,  882.78015, 1147.5598 , 1079.2466 ],
               [ 641.4651 ,  425.7917 ,  836.5166 ,  783.908  ],
               [1234.2535 ,  265.3804 , 1481.3005 ,  739.09204]], dtype=float32)
    keypoint_x_labels: array([[[0.        , 0.        , 0.        , ..., 0.        ,
                 0.        , 0.        ],
                [0.        , 0.        , 0.        , ..., 0.        ,
                 0.        , 0.        ],
                [0.        , 0.        , 0.        , ..., 0.        ,
                 0.        , 0.        ],
                ...,
                [0.        , 0.        , 0.        , ..., 0.        ,
                 0.        , 0.        ],
                [0.        , 0.        , 0.        , ..., 0.        ,
                 0.        , 0.        ],
                [0.        , 0.        , 0.        , ..., 0.        ,
             

# Video Example



In [ ]:
video = "C:\\Users\\mikeG\\Documents\\school\\cisc-452\\CISC-452-Group-22\\cropped\\12_2019-05-03_1.mp4"
output = "output.mp4"
cap = cv2.VideoCapture(video)
video_writer = None
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    data_samples = process_img(frame)

    frame_vis = visualizer.get_image()

    if video_writer is None:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(
            output,
            fourcc,
            25,  # saved fps
            (frame_vis.shape[1], frame_vis.shape[0]))

    video_writer.write(mmcv.rgb2bgr(frame_vis))
video_writer.release()





c:\users\mikeg\documents\school\cisc-452\cisc-452-group-22\mmpose\mmpose\models\heads\coord_cls_heads\rtmcc_head.py:217: UserWarning: The predicted simcc values are normalized for visualization. This may cause discrepancy between the keypoint scores and the 1D heatmaps.
  warnings.warn('The predicted simcc values are normalized for '


: 